In [1]:
import numpy as np 
import cv2
import torch 
from matplotlib import pyplot as plt 
import seaborn as sns 
from Config import Config

> # Creating Custor tensor dataset 


In [2]:
# Import the necessary packages 

from torch.utils.data import Dataset 

class CustorTensorDataset(Dataset): 
    def __init__(self,tensor,transforms=None) -> None:
        super().__init__()
        self.tensor = tensor 
        self.transforms = transforms

    def __getitem__(self, index):
        
        image = self.tensors[0][index]
        label = self.tensors[1][index]
        bbox = self.tensors[2][index]

        image = image.permute(2,0,1)

        if self.transforms: 
            image = self.transforms(image)


        return (image, label, bbox)

    def __len__(self):
		# return the size of the dataset
	    return self.tensors[0].size(0)


> # Building the Objection Detection Architecture


In [ ]:
from torch.nn import Dropout
from torch.nn import Identity
from torch.nn import Linear
from torch.nn import Module
from torch.nn import ReLU
from torch.nn import Sequential
from torch.nn import Sigmoid



class objectDetector(torch.nn.Module): 
    def __init__(self, baseModel, numClasses):
        super(objectDetector,self).__init__()

        self.baseModel = baseModel
        self.numClasses = numClasses 

        self.regressor = Sequential ( 
            Linear(baseModel.fc.in_features, 128),
            ReLU(),
            Linear(128,64),
            ReLU(),
            Linear(64,32), 
            ReLU(), 
            Linear(32,4), 
            Sigmoid()

        )

        # build the classifier head to predict the class labels
        self.classifier = Sequential (
            Linear(baseModel.fc.in_features, 512),
			ReLU(),
			Dropout(),
			Linear(512, 512),
			ReLU(),
			Dropout(),
			Linear(512, self.numClasses)
        )

        self.baseModel = Identity()

    def forward(self, X): 
        # pass the inputs through the base model and then obtain
	    # predictions from two different branches of the network 

        features = self.baseModel(X)
        bboxes = self.regressor(features)
        classLogits = self.classifier(features)

        return (bboxes,classLogits)
